# 1. Dataset
### Import libraries & dataset
First I would like to import some libraries needed and the dataset. <br>
Dataset source: https://archive.ics.uci.edu/ml/datasets/Adult

In [1]:
from google.colab import drive 
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np

df = pd.read_csv('/content/drive/MyDrive/adult.data.csv', header = None)
df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


## 1.1. Dataset summary
The dataset was originally intended for classification tasks and comes pre-split into training and test sets, in files named <b>adult.data</b> and <b>adult.test</b> respectively. The training set contains $N_{training} = 32561$ data tuples and the test set contains $N_{test} = 16281$ tuples. In this project, I have analyzed the 2 sets separately to obtain the results.
<br> Both of the dataset files contain the same 14 attributes: `age`, `workclass`, `fnlwgt`, `education`, `education-num`, `marital-status`, `occupation`, `relationship`, `race`, `sex`, `capital-gain`, `capital-loss`, `hours-per-week`, `native-country` and 1 class label of <b>yearly income</b>. <br> Since this is a frequent itemset mining task rather than classification, I have treated all of the attributes and class labels as <b>items</b>, making all 15 of the information entries conceptually equivalent. 

## 1.2. Data pre-processing
### Rename columns
As you can see, the dataset does not have proper column names so we should give proper names to the columns for convenience.

In [3]:
df.columns = ['age', 'workclass','fnlwgt', 'education', 'education-num',
              'marital-status','occupation', 'relationship','race', 'sex',
              'capital-gain','capital-loss','hours-per-week','native-country',
              'income_category']
df.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income_category
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


### Drop redundant columns
The attribute `education-num` is just a proxy for 'education', so I removed the `education-num` attribute.

In [4]:
df = df.drop("education-num", axis = 1) 
for col in df.columns:
    print(df[col].value_counts())

36    898
31    888
34    886
23    877
35    876
     ... 
83      6
88      3
85      3
86      1
87      1
Name: age, Length: 73, dtype: int64
 Private             22696
 Self-emp-not-inc     2541
 Local-gov            2093
 ?                    1836
 State-gov            1298
 Self-emp-inc         1116
 Federal-gov           960
 Without-pay            14
 Never-worked            7
Name: workclass, dtype: int64
164190    13
203488    13
123011    13
148995    12
121124    12
          ..
232784     1
325573     1
140176     1
318264     1
257302     1
Name: fnlwgt, Length: 21648, dtype: int64
 HS-grad         10501
 Some-college     7291
 Bachelors        5355
 Masters          1723
 Assoc-voc        1382
 11th             1175
 Assoc-acdm       1067
 10th              933
 7th-8th           646
 Prof-school       576
 9th               514
 12th              433
 Doctorate         413
 5th-6th           333
 1st-4th           168
 Preschool          51
Name: education, dtype: int6

Clearly, the attributes `workclass`, `occupation` and `native-country` have blank entries in them, labeled with `?` marks so I decided to treat the `?` entries as <b>categorical values</b> of the respective attributes, since all of the entries are eventually treated as <b>items</b> anyway.
<br> To differentiate between the `?` entries of one attribute from those of others, I concatenated all of the entries of these 3 attributes with their respective attribute names. 
<br> Therefore, the workclass `Private` became `Private_workclass`, the entry `?` became `?_workclass`, so on and so forth for `occupation` and `native-country` as well. 


In [5]:
# Deal with ? entries
cols_with_blank_entries = ["workclass", "occupation", "native-country"] 
for col in cols_with_blank_entries:
    df["converted_" + col] = df[col].astype(str).replace(" ", "") + "_" + col
    df = df.drop(col, axis = 1)

The attribute `age`, `fnlwgt`, `capital-gain`, `capital-loss` and `hours-per-week` are numeric. I split each of them into <b>10 equal-width bins</b> and took the bin boundary information as <b>categorical values</b>. Furthermore, I concatenated these categorical names with the attribute names for clarity. 
<br> Therefore, the age `39` became `age_(38.9, 46.2]`, holding the attribute and binning information.

In [6]:
# Deal with numeric attribures
numeric_cols = ["age", "fnlwgt", "capital-gain", "capital-loss", "hours-per-week"] 
for col in numeric_cols:
    bins = np.histogram(df[col])
    bins = list(bins[1]) # Generate 10 equal-width bins
    if (bins[0] == 0.0):
        bins[0] = -0.1
    category = pd.cut(df[col], bins)
    category = category.to_frame()
    category.columns = ['converted_' + col]
    df = pd.concat([df, category], axis = 1)
    df["converted_" + col] = col + "_" + df["converted_" + col].astype(str).replace(" ", "")
    df = df.drop(col, axis=1)

Finally, I converted the dataset into a dictionary, with keys corresponding to <b>transaction ID's</b> and values corresponding to the lists of <b>items</b> (holding attribute values).
<br> Consequently, every tuple in the original dataset became a (processed) list of items. This dictionary was fed into the code as the input database.

In [7]:
# Convert to dictionary format for using as an input to the program
dict_table = {}
temp_list = []
for index, data in df.iterrows():
    dict_table[str(index)] = data.tolist()

# 3. Implementation
<b><i>Data structure</i></b>: 
- Each <b>item</b> is represented with a <b>string</b>. I have used the <mark>list data structure</mark> in Python to store each itemset. 
- A list of lists is used to store the frequent <i>k</i>-itemsets. Then, all of these <i>k</i>-itemsets are further stored in a list of lists of lists, named <i>L</i>. 
<br> In consequence: 
  - L[0] holds frequent 1-itemsets
  - L[1] holds frequent 2-itemsets and so on. 

- I have also used a list of lists of dictionaries, <i>C</i>, to hold the support information along with the itemsets. 
<br> Hence, C[0] is a list of dictionaries, where each of the dictionaries hold the itemset members and supports of the frequent 1-itemsets. 

In [10]:
import itertools

def generate_counts_C(candidate_C, dict_table):
    temp_C = []
    for candidate_itemset in candidate_C:
        temp_C.append({"itemset": candidate_itemset, "support": 0})
        for dict_list in dict_table.values(): 
            if (all(x in dict_list for x in candidate_itemset)):
                temp_C[-1]["support"] += 1   
    return temp_C

def has_infrequent_subset(candidate_itemset, set_itemset, k): # Make sure all subsets are frequent
    subsets = list(itertools.combinations(candidate_itemset, k-1))
    for x in subsets:
        if (list(x) not in set_itemset):
            return True
    return False

def check_share_condition(itemset_l1, itemset_l2, num_share): # Make sure the first k-2 elements are same
    for x in range(num_share):
        if (itemset_l1[x] != itemset_l2[x]):
            return False
    if (itemset_l1[num_share] >= itemset_l2[num_share]): # Violate -> false
        return False
    return True

def generate_candidate_C(set_itemset, k):
    num_share = k - 2 
    candidate_C = []
    for itemset_l1_index, itemset_l1 in enumerate(set_itemset):
        for itemset_l2_index, itemset_l2 in enumerate(set_itemset):
            if (check_share_condition(itemset_l1, itemset_l2, num_share)): # Join members of L_{k-1} if their first k-2 items are common
                candidate_itemset = sorted(list(set().union(itemset_l1, itemset_l2))) # Join step
                if (has_infrequent_subset(candidate_itemset, set_itemset, k) is False): # Prune step
                    candidate_C.append(candidate_itemset)    
    return candidate_C

def prune(temp_C, min_sup_count):
    pruned_temp_C = [x for x in temp_C if x["support"] >= min_sup_count]
    pruned_temp_L = sorted([x["itemset"] for x in temp_C if x["support"] >= min_sup_count])
    return pruned_temp_L, pruned_temp_C

def generate_counts_C1(dict_table):
    temp_C = []
    items = [item for dict_list in dict_table.values() for item in dict_list]
    values, counts = np.unique(items, return_counts = True)
    for x in range(len(values)):
        temp_C.append({"itemset": [values[x]], "support": counts[x]})
    return temp_C

def apriori(dict_table, support):
    L = []
    C = []
    min_sup_count = len(dict_table) * support
    
    # Generate frequent 1-itemsets. L[0] holds the 1-itemsets, while C[0] holds the 1-itemsets with their support counts
    print("Generating Frequent 1-itemsets")
    counts_C1 = generate_counts_C1(dict_table) # Generates candidates for 1-itemsets and their counts
    pruned_L1, pruned_C1 = prune(counts_C1, min_sup_count) # Pruning against min_sup
    L.append(pruned_L1)
    for x in pruned_C1:
        x['support'] = (float(x['support']) / float(len(dict_table)))
    C.append(pruned_C1)
    
    k = 2
    while(1):
        print("Generating Frequent " + str(k) + "-itemsets")
        candidate_C = generate_candidate_C(L[k-2], k = k) # Same as the 'apriori_gen' procedure in the book
        counts_C = generate_counts_C(candidate_C, dict_table) # Find counts
        pruned_L, pruned_C = prune(counts_C, min_sup_count) # Prune if support condition is not met
        
        if (not pruned_L): # Break if pruned L is an empty set
            break
        
        L.append(pruned_L)
        for x in pruned_C:
            x['support'] = (float(x['support']) / float(len(dict_table)))
        C.append(pruned_C)
        k = k + 1
    return L, C

In [11]:
import json 

if __name__ == '__main__':
    support = 0.45
    L, C = apriori(dict_table, support = support)
    num_of_frequent_itemsets = sum(len(x) for x in L)
    print(" ")
    print("Number of frequent itemsets:")
    print(num_of_frequent_itemsets)
    print(" ")
    print("Frequent itemsets with support: ")
    print(json.dumps(C, indent=4))

Generating Frequent 1-itemsets
Generating Frequent 2-itemsets
Generating Frequent 3-itemsets
Generating Frequent 4-itemsets
Generating Frequent 5-itemsets
Generating Frequent 6-itemsets
 
Number of frequent itemsets:
88
 
Frequent itemsets with support: 
[
    [
        {
            "itemset": [
                " <=50K"
            ],
            "support": 0.7591904425539756
        },
        {
            "itemset": [
                " Male"
            ],
            "support": 0.6692054912318418
        },
        {
            "itemset": [
                " Married-civ-spouse"
            ],
            "support": 0.4599367341297872
        },
        {
            "itemset": [
                " Private_workclass"
            ],
            "support": 0.6970301894904948
        },
        {
            "itemset": [
                " United-States_native-country"
            ],
            "support": 0.895857006848684
        },
        {
            "itemset": [
                